In [1]:
import pandas as pd
import re

In [2]:
data=pd.read_excel("C:/Users/RADHANI1/OneDrive - Novartis Pharma AG/Desktop/SML/HawkAI/Sprint4/POS_all_molecules3.xlsx")

In [3]:
data.head(2)

Unnamed: 0       Molecule Scenarios   NCT Number  \
0           0  pembrolizumab    cancer  NCT05361915   
1           1  pembrolizumab    cancer  NCT05468034   

                                               Title Acronym  \
0  Study to Assess Abivertinib in Combination Wit...     NaN   
1       Exercise in Metastatic Breast Cancer: EMBody     NaN   

               Status         Study Results  \
0  Not yet recruiting  No Results Available   
1  Not yet recruiting  No Results Available   

                                        Conditions  \
0       Prostate Cancer|Prostate Cancer Metastatic   
1  Breast Cancer|Indolent Metastatic Breast Cancer   

                         Interventions  ...        Other IDs    Start Date  \
0  Drug: Abivertinib|Drug: Abiraterone  ...      ABV-RPC-201  October 2022   
1    Behavioral: Exercise Intervention  ...  CTO-IUSCCC-0781   August 2022   

  Primary Completion Date Completion Date   First Posted  \
0            January 2028    January 2028    May 5, 2022   
1           December 2027   December 2027  July 21, 2022   

   Results First Posted Last Update Posted  \
0                   NaN        May 5, 2022   
1                   NaN      July 21, 2022   

                                           Locations Study Documents  \
0  UCSD Moores Cancer Center, La Jolla, Californi...             NaN   
1  IU Health West, Avon, Indiana, United States|I...             NaN   

                                           URL  
0  https://ClinicalTrials.gov/show/NCT05361915  
1  https://ClinicalTrials.gov/show/NCT05468034  

[2 rows x 29 columns]

### Extract Phases

In [4]:
#Extract Phase by rules
data['Phases_modified'] = ""
for ind in range(0,len(data['Phases'])):
#     print(ind)
#     print(data['Phases'][ind])
    if data['Phases'][ind] == "Phase 1|Phase 2":
        data['Phases_modified'][ind] = "Phase 1"
    elif data['Phases'][ind] == "Phase 2|Phase 3":
        data['Phases_modified'][ind] = "Phase 3"
    elif data['Phases'][ind] == "Early Phase 1":
        data['Phases_modified'][ind] = "Phase 1"
    elif data['Phases'][ind] == "Not Applicable":
        data['Phases_modified'][ind] = None
    else:
        data['Phases_modified'][ind] = data['Phases'][ind]
#     print(data['Phases_modified'][ind])

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\470248257.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Phases_modified'][ind] = data['Phases'][ind]
C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\470248257.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Phases_modified'][ind] = ""
C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\470248257.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat

### Extract Countries

In [5]:
# Function to extract countries
data['Countries'] = ""
def extract_country(text):
    country_list = []
    loc_list = str(text).split("|")
    for loc_item in loc_list:
        location_list = loc_item.split(",")
        country_list.append((location_list[-1]).strip())
#     country_string = ",".join(list(np.unique(country_list)))
    country_string = ",".join((country_list))
    return country_string

In [6]:
#Extracting countries from all the rows
for ind in range(0,len(data['Locations'])):
    data['Countries'][ind] = extract_country(data['Locations'][ind])
data = data.replace("nan", "") 

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\3301314259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Countries'][ind] = extract_country(data['Locations'][ind])


### Extract Competing drugs

In [7]:
data['Drug_Bilogical'] = ""
for mol_ind in range(0,len(data['Molecule'])):
    intervention_list = str(data['Interventions'][mol_ind]).split('|')
    drug_biology_list = []
    for drug_ind in range(0,len(intervention_list)):
        if ("Drug:" in intervention_list[drug_ind]):
            drug_biology_list.append(re.sub("Drug:", "", intervention_list[drug_ind]))
        elif ("Biological:" in intervention_list[drug_ind]):
            drug_biology_list.append(re.sub("Biological:", "", intervention_list[drug_ind]))
    data['Drug_Bilogical'][mol_ind] = ",".join(drug_biology_list)           

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\212650173.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Drug_Bilogical'][mol_ind] = ",".join(drug_biology_list)


In [8]:
data['Competing_Drugs'] = ""
for mol_ind in range(0,len(data['Molecule'])):
    drug_list = str(data['Drug_Bilogical'][mol_ind]).split(',')
    competing_drug_list = []
    for drug_ind in range(0,len(drug_list)):
        if not(data['Molecule'][mol_ind].lower() in drug_list[drug_ind].lower()):
            competing_drug_list.append(drug_list[drug_ind].lower())
        elif "and" in drug_list[drug_ind].lower():
            drug_list_and = drug_list[drug_ind].lower().split('and')
            for drug_and_ind in range(0,len(drug_list_and)):
                if not(data['Molecule'][mol_ind].lower() in drug_list_and[drug_and_ind].lower()):
                    competing_drug_list.append(drug_list_and[drug_and_ind].lower())
    if not('nan' in competing_drug_list):
        competing_drug_string = ','.join(competing_drug_list)
    data['Competing_Drugs'][mol_ind] = competing_drug_string
    competing_drug_string = ""
#     print(competing_drug_string)

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\2203720410.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Competing_Drugs'][mol_ind] = competing_drug_string


### Extract Competing drugs with main, without main and Presence of main drug

In [9]:
data['Competing_Drugs_with_main'] = ""
data['Competing_Drugs_without_main'] = ""
data['Presence of Main Drug'] = ""

In [10]:
def flatten_list(lst):
    new_list = []
    for i in range(0,len(lst)):
        if type(lst[i]) == list:
            for j in lst[i]:
                new_list.append(j)   
        else:
            new_list.append(lst[i])
    return new_list      

In [11]:
# Extract combination drugs with main molecule
filler_list = [" and "," plus ","\+","combination regimen","a combination group of","combination of three","combined with","in combination","in conjugation"," with "]
def extract_drug(text):
    inter_list = text.split("|")
    drug_list = []
    for inter in inter_list:
        if re.search("^Combination\sProduct\:", inter):
            drug_list.append(re.sub("^Combination Product\:", "", inter))
        elif re.search("^Drug\:", inter):
            drug_list.append(re.sub("^Drug\:", "", inter))
        elif re.search("^Biological\:", inter):
            drug_list.append(re.sub("^Biological\:", "", inter))
        else:
            for filler in filler_list:
                if filler.lower() in inter.lower():
                    drug_item = (re.sub(filler.lower(), ",", inter.lower())).strip()
                    drug_item = (re.sub(".*:", ",", inter.lower())).strip()
                    drug_item = drug_item.replace("+", ",")
                    drug_item = drug_item.replace("/", ",")     
                    drug_list.append(drug_item)                           
                else:
                    drug_item = re.findall("(.*?):", inter)
                    drug_list.append(drug_item)
                    drug_list = flatten_list(drug_list)
    drug_list = list(set(drug_list))
    drug_string = ",".join(drug_list)
    return drug_string

In [12]:
# Extract competing drugs with and without main
for ind in range(0,len(data)):
    data['Competing_Drugs_with_main'][ind] = extract_drug(str(data['Interventions'][ind])).strip(",")
    if data['Molecule'][ind].lower() in data['Competing_Drugs_with_main'][ind].lower():
        data['Competing_Drugs_without_main'][ind] = (re.sub(data['Molecule'][ind].lower(), "", data['Competing_Drugs_with_main'][ind].lower()))
        data['Competing_Drugs_without_main'][ind] = (re.sub(", ,", ",", data['Competing_Drugs_without_main'][ind])).strip(",")
    else:
        data['Competing_Drugs_without_main'][ind] = data['Competing_Drugs_with_main'][ind].strip(",")
    if data['Molecule'][ind].lower() in str(data['Interventions'][ind]).lower():
        data['Presence of Main Drug'][ind] = 'yes'
    else:
        data['Presence of Main Drug'][ind] = 'no'

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\1809564521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Competing_Drugs_with_main'][ind] = extract_drug(str(data['Interventions'][ind])).strip(",")
C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\1809564521.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Competing_Drugs_without_main'][ind] = data['Competing_Drugs_with_main'][ind].strip(",")
C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\1809564521.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

### Extract Combination drugs

In [13]:
data['Combination drugs raw'] = ""

In [14]:
#Extracting texts with combination drugs
for mol_ind in range(0,len(data['Molecule'])):
    ele_list = str(data['Interventions'][mol_ind]).split('|')
    combination_list = []
    for ele_ind in range(0,len(ele_list)):
        if "Combination Product:" in ele_list[ele_ind]:
            combination_list.append((re.sub("Combination Product:", "", ele_list[ele_ind])).strip())
        else:
            for filler in ["comb"," plus "," with ","+"," and "]:
                if filler in ele_list[ele_ind].lower():
                    combination_list.append(ele_list[ele_ind])
        if not combination_list == []:
            data['Combination drugs raw'][mol_ind] = combination_list

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\3304345055.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Combination drugs raw'][mol_ind] = combination_list


In [15]:
#Extracting combination drugs
filler_list = [" and "," plus ","\+","combination regimen","a combination group of","combination of three","combined with","in combination","in conjugation"," with "]
def clean_text(text_list):
    cleantext_list = []
    cleantext_string = ""
    for text in text_list:
#         print(text)
        for filler in filler_list:
            if filler.lower() in text.lower():
                text = (re.sub(filler.lower(), ",", text.lower())).strip()
        if not text in cleantext_list:
            cleantext_list.append(text)
        if cleantext_list == []:
            cleantext_list = text_list
        cleantext_string = ','.join(cleantext_list)
    return cleantext_string  

In [16]:
# Cleaning symbols representing combination drugs
def clean_symbols(text):
    cleantext_list = []
    cleantext_string = ""
    for sym in ["+","/"]:
        if sym in text.lower():
            text = text.replace(sym, ",")
#         if not text in cleantext_list:
#             cleantext_list.append(text)
#         if cleantext_list == []:
#             cleantext_list = text_list
#         cleantext_string = ','.join(cleantext_list)
    return text  

In [17]:
data['Combination_drugs_raw'] = ""

In [18]:
for drug_ind in range(0,len(data['Molecule'])):
    data['Combination_drugs_raw'][drug_ind] = clean_text(data['Combination drugs raw'][drug_ind])
    data['Combination_drugs_raw'][drug_ind] = clean_symbols(data['Combination_drugs_raw'][drug_ind])

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\3923959207.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Combination_drugs_raw'][drug_ind] = clean_text(data['Combination drugs raw'][drug_ind])
C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\3923959207.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Combination_drugs_raw'][drug_ind] = clean_symbols(data['Combination_drugs_raw'][drug_ind])


In [19]:
data['Combination drugs'] = ""
#Removing titles
filler_list = ["Procedure:","Drug:","Radiation:","Other:","Biological:","Diagnostic Test:"]
for i in range(0,len(data['Molecule'])):
    drug_list = []
    drug_text = data['Combination_drugs_raw'][i]
    for k in filler_list:
        if k.lower() in drug_text.lower():
            drug_text = (re.sub(k.lower(), "", drug_text.lower()).strip())
#             print(drug_text)
    if not drug_text in drug_list:
        drug_list.append(drug_text)
    data['Combination drugs'][i] = ",".join(drug_list)              

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\4292511504.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Combination drugs'][i] = ",".join(drug_list)


In [20]:
# Removing duplicates
for drug_ind in range(0,len(data['Molecule'])):
    comb_drug = ",".join(list(set(data['Combination drugs'][drug_ind].split(",")))) 
    data['Combination drugs'][drug_ind] = comb_drug.strip(",")

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\2973484463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Combination drugs'][drug_ind] = comb_drug.strip(",")


In [21]:
data = data.drop(['Combination drugs raw','Combination_drugs_raw','Drug_Bilogical'], axis=1)

### Is Combination drugs field

In [22]:
data['is_combination'] = ""
for index in range(0,len(data['Combination drugs'])):
    if data['Combination drugs'][index] == "":
        data['is_combination'][index] = 'no'
    else:
        data['is_combination'][index] = 'yes'     

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\4280915878.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_combination'][index] = 'no'
C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\4280915878.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_combination'][index] = 'yes'


In [23]:
data

Unnamed: 0       Molecule    Scenarios   NCT Number  \
0               0  pembrolizumab       cancer  NCT05361915   
1               1  pembrolizumab       cancer  NCT05468034   
2               2  pembrolizumab       cancer  NCT03818087   
3               3  pembrolizumab       cancer  NCT04178889   
4               4  pembrolizumab       cancer  NCT05504707   
...           ...            ...          ...          ...   
11566       11566  pembrolizumab  lung cancer  NCT04672434   
11567       11567  pembrolizumab  lung cancer  NCT04888806   
11568       11568  pembrolizumab  lung cancer  NCT05279378   
11569       11569  pembrolizumab  lung cancer  NCT03568019   
11570       11570  pembrolizumab  lung cancer  NCT05268952   

                                                   Title   Acronym  \
0      Study to Assess Abivertinib in Combination Wit...       NaN   
1           Exercise in Metastatic Breast Cancer: EMBody       NaN   
2       Elevate! : An Elderly Breast Cancer Cohort Study       NaN   
3        Second Primary Lung Cancer Cohort Study (SPORT)     SPORT   
4      DecipHER Trial - DC1 Tx for Early-Stage TNBC a...  DecipHER   
...                                                  ...       ...   
11566  Sym024 Monotherapy and in Combination With Sym...       NaN   
11567  A Trial of Camrelizumab Combined With Microwav...       NaN   
11568  Correlation of Lung Ultrasonography With Chest...       NaN   
11569  PET Avidity in Cachexia-Inducing Lung and Gast...       NaN   
11570  A Study to Evaluate the Value of Circulating T...       NaN   

                       Status         Study Results  \
0          Not yet recruiting  No Results Available   
1          Not yet recruiting  No Results Available   
2                  Recruiting  No Results Available   
3                  Recruiting  No Results Available   
4                  Recruiting  No Results Available   
...                       ...                   ...   
11566              Recruiting  No Results Available   
11567      Not yet recruiting  No Results Available   
11568              Recruiting  No Results Available   
11569  Active, not recruiting  No Results Available   
11570              Recruiting  No Results Available   

                                              Conditions  \
0             Prostate Cancer|Prostate Cancer Metastatic   
1        Breast Cancer|Indolent Metastatic Breast Cancer   
2                                          Breast Cancer   
3                                            Lung Cancer   
4      Triple Negative Breast Cancer|HER2-negative Br...   
...                                                  ...   
11566                      Metastatic Cancer|Solid Tumor   
11567  Colorectal Cancer Metastatic|Liver Metastases|...   
11568                                    Cancer Patients   
11569                                   Cachexia; Cancer   
11570                              Neuroendocrine Tumors   

                                           Interventions  ... Study Documents  \
0                    Drug: Abivertinib|Drug: Abiraterone  ...             NaN   
1                      Behavioral: Exercise Intervention  ...             NaN   
2                       Behavioral: Observational cohort  ...             NaN   
3                                                    NaN  ...             NaN   
4      Biological: HER2 - primed Dendritic cells|Biol...  ...             NaN   
...                                                  ...  ...             ...   
11566                          Drug: Sym021|Drug: Sym024  ...             NaN   
11567  Drug: Camrelizumab|Procedure: ablation|Drug: c...  ...             NaN   
11568          Radiation: lung ultrasonography versus CT  ...             NaN   
11569                         Other: Retrospective study  ...             NaN   
11570  Other: Liquid biopsies|Other: Scans (CT, galli...  ...             NaN   

                                               URL Phases_modified

### Export final data

In [24]:
#Function to write the data to the excel
export_path = "C:/Users/RADHANI1/OneDrive - Novartis Pharma AG/Desktop/SML/HawkAI/Sprint4/POS_all_molecules_final_3.xlsx"
def write_output(data):
    writer = pd.ExcelWriter(export_path, engine='xlsxwriter',options={'strings_to_urls': False})
    data.to_excel(writer, index=False)

    writer.save()
    writer.close()

In [25]:
write_output(data)

C:\Users\RADHANI1\AppData\Local\Temp\1\ipykernel_3328\2156260384.py:4: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(export_path, engine='xlsxwriter',options={'strings_to_urls': False})
C:\Users\RADHANI1\.conda\envs\HawkAI\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
